In [8]:
# Library import
import sys
sys.path.append('../Library/') 
import datetime as dt
import get_data as gd
import pandas as pd

In [ ]:
# init db engine
ods = gd.Get_Data(
    'postgres',
    'HRODS',
    'DEI')

In [ ]:
# set engine
engine = ods.set_engine()

#set paths
ods.set_output_path("C:\\Users\\garciand\\OneDrive - HP Inc\\Desktop\\Deliverables\\DEI\\")



In [ ]:
# query
query = """
Select 

    1 As "Headcount",
    CASE
        WHEN wd."Pay Group Country Desc" IN (
        'Austria',
        'Belgium',
        'Bulgaria',
        'Croatia',
        'Czechia',
        'Denmark',
        'FinlAnd',
        'Greece',
        'Hungary',
        'IrelAnd',
        'Israel',
        'Kazakhstan',
        'Luxembourg',
        'Morocco',
        'NetherlAnds',
        'Nigeria',
        'Norway',
        'PolAnd',
        'Portugal',
        'Russian FederatiOn',
        'Saudi Arabia',
        'Serbia',
        'Slovakia',
        'South Africa',
        'Sweden',
        'Tunisia',
        'Turkey',
        'United Arab Emirates') THEN 'U'
        WHEN pwd."Employee ID" IS NULL THEN 'N'
        ELSE pwd."Is PWD?"
	END AS "PWD Status",
	
    CASE
        WHEN WD."Pay Group Country Code" = N'USA' THEN WD."Pay Group Country Desc"
        ELSE 'Other'
    END AS "Pay Group Country Desc"     

From
    "HPW_DAILY" wd
Left Join 
    "HPW_W_DISABILITIES" AS pwd ON pwd."Employee ID" = wd."Worker ID" AND pwd."Report Date" = (Select Max("Report Date") From "HPW_W_DISABILITIES")
Where 
    wd."Worker Reg / Temp Code" = 'R'
And 
    wd."Worker Status Category Code" = 'A'
And
    wd."Report Date" = (Select Max("Report Date") From "HPW_DAILY")
"""

In [ ]:
data = ods.internal_query(query, engine)

# Headcount
ww_headcount = data["Headcount"].sum()
us_headcount = data.loc[data["Pay Group Country Desc"] == "United States of America"][
    "Headcount"
].sum()

# people with disabilities
ww_pwd_headcount = data.loc[data["PWD Status"] == "Y"]["Headcount"].sum()
us_pwd_headcount = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["PWD Status"] == "Y")
]["Headcount"].sum()

# % people with disabilities
ww_pwd_headcount_p = round((ww_pwd_headcount / ww_headcount) * 100, 1).astype(str) + "%"
us_pwd_headcount_p = round((us_pwd_headcount / us_headcount) * 100, 1).astype(str) + "%"


In [ ]:
# data to dataframe
kpi = [
    "• WW + U.S. Total employment in-scope",
    "• WW + U.S. Number of employees with disabilities",
    "• WW + U.S. Percentage of employees with disabilities",
]

kpi_data = [
    "WW: " + str(ww_headcount) + ", US: " + str(us_headcount),
    "WW: " + str(ww_pwd_headcount) + ", US: " + str(us_pwd_headcount),
    "WW: " + str(ww_pwd_headcount_p) + ", US: " + str(us_pwd_headcount_p),
]

kpi_to_list = list(zip(kpi, kpi_data))

# Converting lists of tuples into
# pandas Dataframe.
df = pd.DataFrame(kpi_to_list, columns=["Metric", "Result"])

# Print data.
df


,Metric,Result
0,• WW + U.S. Total employment in-scope,"WW: 50583, US: 11745"
1,• WW + U.S. Number of employees with disabilities,"WW: 293, US: 80"
2,• WW + U.S. Percentage of employees with disab...,"WW: 0.6%, US: 0.7%"


In [ ]:
ods.export_data(df,"WBA","Yes")